In [ ]:
!python ../setup.py build 
!python ../setup.py build_ext --inplace

In [1]:
import os, sys
sys.path.append("..")

# Optimization of Pauli Poly

There are two measure to estimate performances of `PauliPoly`.

- Naive `PauliPoly` implementation.
- Qiskit `SparsePauliOp` and `PauliList`.
- 
The PauliPoly in the origial OptTrot library was a naive implementation written by python.
In here, we will optimize the routine as soon as possible using scipy and numba routines.

In [2]:
import opttrot

In [3]:
from opttrot.pauli_c import get_paulilist_from_coefs

In [4]:
from opttrot.pauli_c import PauliElement

In [5]:
from opttrot.pauli import PauliElement
from opttrot.pauli import PauliPoly

import numpy as np
import matplotlib.pyplot as plt

In [6]:
from importlib import reload
#import paulipoly
#reload(paulipoly)
#from paulipoly import PauliPoly as NewPauliPoly
#import ten_con
#reload(ten_con)
#from ten_con import _mat_to_coef_mat, _coef_to_mat

In [7]:
from scipy.sparse import coo_matrix, csr_matrix

In [8]:
from qiskit.quantum_info import Pauli, PauliList, SparsePauliOp

In [9]:
from copy import deepcopy

In [10]:
qubits = 4
def get_test_hermit(qubits, tol= 0.5):
    ndim = int(2**qubits)
    A = np.random.rand(ndim, ndim)
    A[A<tol] = 0.
    H = A + 1j*A
    H = H.conj().T@H
    return H


In [11]:
A = np.sqrt(get_test_hermit(qubits, 0.96))
B = np.sqrt(get_test_hermit(qubits, 0.8))


In [12]:
test_ppoly = PauliPoly.from_matrix(A)

In [13]:
test_ppoly.poly

array([PauliElement(n=4, weight=1.115432+(-0.000000)j, IIII),
       PauliElement(n=4, weight=0.173188+(0.000000)j, IIIX),
       PauliElement(n=4, weight=0.173370+(-0.000000)j, IIXX),
       PauliElement(n=4, weight=0.344493+(-0.000000)j, IXII),
       PauliElement(n=4, weight=0.173568+(0.000000)j, IXXX),
       PauliElement(n=4, weight=0.174622+(-0.000000)j, XIXI),
       PauliElement(n=4, weight=0.173866+(0.000000)j, XIXX),
       PauliElement(n=4, weight=-0.180697+(-0.000000)j, IIIZ),
       PauliElement(n=4, weight=-0.344493+(0.000000)j, IXIZ),
       PauliElement(n=4, weight=-0.000000+(0.000000)j, IXXY),
       PauliElement(n=4, weight=-0.000000+(0.000000)j, XIXY),
       PauliElement(n=4, weight=-0.174622+(0.000000)j, XIXZ),
       PauliElement(n=4, weight=-0.223002+(-0.000000)j, IIZI),
       PauliElement(n=4, weight=-0.173188+(0.000000)j, IIZX),
       PauliElement(n=4, weight=-0.000000+(0.000000)j, IXYX),
       PauliElement(n=4, weight=0.000081+(0.000000)j, IXZI),
       Pau

In [14]:
#plist = list(test_ppoly._terms.values())
coef_mat = test_ppoly.coef_matrix
mat = test_ppoly.matrix
coef_mat[coef_mat<0.05] = 0.
cmat = csr_matrix(coef_mat)
100*cmat.data.size/(4**qubits)

9.765625

## Coef matrix to PauliPoly

In [15]:
%%timeit
PauliPoly(coef_mat)

13.2 µs ± 263 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Hermit matrix to PauliPoly

In [16]:
%%timeit
PauliPoly.from_matrix(mat)

71.3 µs ± 513 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
%%timeit
SparsePauliOp.from_operator(mat)

56.4 µs ± 129 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Test

## PauliPoly to Hermit

In [22]:
# PauliPoly
pp001 = PauliPoly.from_matrix(A)
pp002 = PauliPoly.from_matrix(B)

# Qiskit
spa001 = SparsePauliOp.from_operator(A)
spa002 = SparsePauliOp.from_operator(B)

In [23]:
%%timeit
pp001.matrix

45.3 µs ± 2.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
%%timeit
spa001.to_matrix() 

90 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Tset: Qubit 12 (Matrix construction)

- PauliPoly: 3.48 s ± 12.3 ms
- Qiskit: 33.2 s ± 1.98 s

In [25]:
%%timeit
pp002.matrix

44 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%%timeit
spa002.to_matrix()

96.8 µs ± 561 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## PauliElement generation

In [27]:
ps = pp001.poly

In [28]:
ps

array([PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIII),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIIX),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIXX),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IXII),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IXXX),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, XIXI),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, XIXX),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIIY),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIIZ),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIXY),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IXIZ),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IXXY),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, XIXY),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, XIXZ),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j, IIYX),
       PauliElement(n=4, weight=-nan(ind)+(-nan(ind))j,

In [29]:
pp001.coef_matrix

matrix([[nan+nanj, nan+nanj,  0. +0.j, nan+nanj, nan+nanj,  0. +0.j,
          0. +0.j, nan+nanj,  0. +0.j,  0. +0.j, nan+nanj, nan+nanj,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [nan+nanj, nan+nanj, nan+nanj,  0. +0.j,  0. +0.j, nan+nanj,
         nan+nanj,  0. +0.j,  0. +0.j,  0. +0.j, nan+nanj, nan+nanj,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [ 0. +0.j, nan+nanj, nan+nanj, nan+nanj,  0. +0.j, nan+nanj,
         nan+nanj,  0. +0.j, nan+nanj, nan+nanj,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [nan+nanj,  0. +0.j, nan+nanj, nan+nanj, nan+nanj,  0. +0.j,
          0. +0.j, nan+nanj, nan+nanj, nan+nanj,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j],
        [nan+nanj,  0. +0.j,  0. +0.j, nan+nanj, nan+nanj, nan+nanj,
          0. +0.j, nan+nanj,  0. +0.j,  0. +0.j,  0. +0.j,  0. +0.j,
          0. +0.j,  0. +0.j, nan+nanj, nan+nanj],
        [ 0. +0.j, nan+nanj, nan+nanj,  0. +0.j, nan+nanj, n

## Addition

**Comparsion: Matrix and set methods**

Qubits = 5

- Matrix: 58.3 ms ± 2.4 ms
- Set: 13.2 ms ± 111 µs

Qubits = 8

- Matrix: 3.68 s ± 67 ms per loop
- Set: 2 s ± 74.5 ms

In [30]:
%%timeit
pp001+pp002

18.3 µs ± 279 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
%%timeit
spa001+spa002

40 µs ± 422 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Mat multiplication

In [12]:
%%timeit
pp001@pp002

NameError: name 'pp001' is not defined

In [36]:
%%timeit
spa001@spa002 # Unable to multiplicate.

138 ms ± 5.14 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Summary

A(xN.N): B took N.N times more slower than A.

|Opeartion|Faster|
|:-------:|:----:|
|o->mat   | Qiskit(x1.2) | 
|mat->o   | PauliPoly(x2-1.2) | 
|Addition | PauliPoly(x2) |
|Mat Mul  | PauliPoly(x Exponential) |
